In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spp.convex_sets import Singleton, Polyhedron
from spp.convex_functions import TwoNorm
from spp.graph import GraphOfConvexSets
from spp.shortest_path import ShortestPathProblem

In [ ]:
# vertices
sets = [
    Singleton([0, 0]),
    Singleton([0, 2]),
    Singleton([0, -2-1e-9]),
    Polyhedron.from_vertices([[2, -2.1], [4, -2.1], [4, 2.1], [2, 2.1]]),
    Singleton([5, 0])
]
sets[3]._center = np.array([3, 0])
vertices = ['s', '1', '2', '3', 't']

# add convex sets to the graph
G = GraphOfConvexSets()
G.add_sets(sets, vertices)
G.set_source('s')
G.set_target('t')

# edges
H = np.hstack((np.eye(2), -np.eye(2)))
l = TwoNorm(H)
edges = {
    's': ('1', '2'),
    '1': ('3',),
    '2': ('3',),
    '3': ('t',),
}
for u, vs in edges.items():
    for v in vs:
        G.add_edge(u, v, l)

# Plot solution of MICP

In [ ]:
spp = ShortestPathProblem(G, relaxation=0, cyclic=False)
sol = spp.solve()
phi = sol.primal.phi
x = sol.primal.x

print('Cost:', sol.cost)
print('\nFlows:')
for k, edge in enumerate(G.edges):
    flow = round(abs(phi[k]), 4)
    print(edge, flow)

In [ ]:
plt.figure(figsize=(3.5,3.5))
G.draw_sets()
G.draw_edges()

plt.text(0.2, 0, r'$s$', ha='left', va='center')
plt.text(0.3, 2, r'$1$', ha='left', va='center')
plt.text(0.3, -2, r'$2$', ha='left', va='center')
plt.text(3, 0.1, r'$3$', ha='center', va='bottom')
plt.text(5, 0.1, r'$t$', ha='center', va='bottom')

kwargs = {'marker': 'o', 'markeredgecolor': 'k', 'markerfacecolor': 'w'}
G.draw_path(phi, x, color='orangered')

plt.grid()
plt.savefig('symmetry_mip.pdf', bbox_inches='tight')

# Plot solution of convex relaxation

In [ ]:
spp = ShortestPathProblem(G, relaxation=1)
sol = spp.solve()
phi = sol.primal.phi
x = sol.primal.x
y = sol.primal.y
z = sol.primal.z

print('Cost:', sol.cost)
print('\nFlows:')
for k, edge in enumerate(G.edges):
    flow = round(abs(phi[k]), 4)
    print(edge, flow)

In [ ]:
import matplotlib
# matplotlib.rc('text', usetex=True)
# matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{bm}'

plt.figure(figsize=(3.5,3.5))
G.draw_sets()

Z = r'\bar{\bf{z}}'
plt.text(.2, 0,
         '$' + Z + '_{(s,1)} =' + Z + '_{(s,2)}$',
         ha='left', va='center')
plt.text(.1, 1.9,
         '$' + Z + '_{(s,1)}^\prime =' + Z + '_{(1,3)}$',
         ha='left', va='top')
plt.text(.1, -1.9,
         r'$' + Z + '_{(s,2)}^\prime =' + Z + '_{(2,3)}$',
         ha='left', va='bottom')
plt.text(3, 1.9,
         r'$' + Z + '_{(1,3)}^\prime$',
         ha='center', va='top')
plt.text(3, -1.9,
         r'$' + Z + '_{(2,3)}^\prime$',
         ha='center', va='bottom')
plt.text(3, 0.1,
         r'$' + Z + '_{(3,t)}$',
         ha='center', va='bottom')
plt.text(5, .1,
         r'$' + Z + '_{(3,t)}^\prime$',
         ha='right', va='bottom')

plt.text(.1, 1, r'$1/2$', ha='left', va='center', c='b')
plt.text(.1, -1, r'$1/2$', ha='left', va='center', c='b')
plt.text(1, 1.99, r'$1/2$', ha='center', va='bottom', c='b')
plt.text(1, -2.05, r'$1/2$', ha='center', va='top', c='b')
plt.text(3.8, 0, r'$1$', ha='right', va='bottom', c='b')

y3t = y[4] / phi[4]
z13 = z[2] / phi[2]
z23 = z[3] / phi[3]

plt.plot([0, 0], [0, 2], 'orangered', **kwargs)
plt.plot([0, 0], [0, -2], 'orangered', **kwargs)
plt.plot([0, z13[0]], [2, z13[1]], 'orangered', **kwargs)
plt.plot([0, z23[0]], [-2, z23[1]], 'orangered', **kwargs)
plt.plot([y3t[0], 5], [y3t[1], 0], 'orangered', **kwargs)

plt.grid()
plt.savefig('symmetry_relaxation.pdf', bbox_inches='tight')